In [73]:
import pandas as pd

train_set_labels = pd.read_csv("training_set_labels.csv")
train_set_features = pd.read_csv("training_set_features.csv")
test_set_features = pd.read_csv("test_set_features.csv")

In [74]:
train_data = train_set_features.merge(train_set_labels, on='respondent_id', how='inner')

In [75]:
train_data = train_data.dropna()

In [76]:
train_data = train_data.dropna()

In [77]:
numerical_features = ['xyz_concern', 'xyz_knowledge']
binary_features = [col for col in train_data.columns if 'behavioral' in col or 'doctor_recc' in col or 'chronic_med_condition' in col or 'child_under_6_months' in col or 'health_worker' in col or 'health_insurance' in col]
categorical_features = [col for col in train_data.columns if col not in ['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'] + numerical_features + binary_features]

In [79]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Preprocess numerical features
numerical_transformer = StandardScaler()
numerical_transformer.fit(train_data[numerical_features])

# Preprocess categorical features
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
categorical_transformer.fit(train_data[categorical_features])

# Impute missing values in binary features
binary_imputer = SimpleImputer(strategy='most_frequent')
binary_imputer.fit(train_data[binary_features])

# Combine the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', binary_imputer, binary_features)
    ])

# Apply the preprocessing to the training data
X_train = preprocessor.fit_transform(train_data.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine']))
y_train = train_data[['xyz_vaccine', 'seasonal_vaccine']]

In [121]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate


In [123]:
from sklearn.linear_model import LogisticRegression
clf = MultiOutputClassifier(RandomForestClassifier(random_state=42))
scoring = ['roc_auc']
scores = cross_validate(clf, X_train, y_train, cv=10, scoring=scoring)

In [124]:
# Print the mean ROC AUC scores
print(f"Mean ROC AUC for xyz_vaccine: {scores['test_roc_auc'][0]:.3f}")
print(f"Mean ROC AUC for seasonal_vaccine: {scores['test_roc_auc'][1]:.3f}")
print(f"Mean ROC AUC: {(scores['test_roc_auc'][0] + scores['test_roc_auc'][1]) / 2:.3f}")

Mean ROC AUC for xyz_vaccine: 0.871
Mean ROC AUC for seasonal_vaccine: 0.847
Mean ROC AUC: 0.859


In [125]:
clf.fit(X_train, y_train)
# Preprocess the test set
test_set_features = test_set_features.fillna(test_set_features.mode().iloc[0])
X_test_final = preprocessor.transform(test_set_features.drop(columns=['respondent_id']))

# Make predictions on the test set
y_test_pred = clf.predict_proba(X_test_final)
y_test_pred_xyz = y_test_pred[0][:, 1]
y_test_pred_seasonal = y_test_pred[1][:, 1]

# Prepare the submission file
submission = pd.DataFrame({
    'respondent_id': test_set_features['respondent_id'],
    'xyz_vaccine': y_test_pred_xyz,
    'seasonal_vaccine': y_test_pred_seasonal
})

# Save to CSV
submission.to_csv('submission_cv_randomforest.csv', index=False)